In [18]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from math import ceil

from itertools import product
import gc
from dateutil import parser

In [19]:
torch.cuda.set_device(0)

In [20]:
import fastai; fastai.__file__

'/home/paperspace/fastai/fastai/__init__.py'

In [21]:
PATH = "data/"
sz=224
arch=resnext101_64
batch_size=5

# Exploring Other Options

In [25]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
sales = pd.read_csv(f'{PATH}sales_train.csv')
sales_test = pd.read_csv(f'{PATH}test.csv')

In [26]:
index_cols  = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'},'item_price': np.mean})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
# all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
all_data = pd.merge(grid,gb,how='left',on=index_cols)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

del grid,gb,cur_shops,cur_items
del sales
gc.collect()

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


580

In [27]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

all_data.fillna(0.0,inplace=True)
all_data['target'] = all_data['target'].clip(0.0,20.0)
all_data['month'] = all_data['date_block_num'] % 12
all_data['year'] = all_data['date_block_num'] // 12

sales_test['date_block_num'] = 34
sales_test['month'] = sales_test['date_block_num']%12
sales_test['year'] = sales_test['date_block_num']//12

all_data = pd.concat([all_data,sales_test])

del sales_test
gc.collect()

49

In [28]:
all_data = all_data.merge(items, on="item_id")
all_data.drop('item_name',axis = 1,inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

all_data = downcast_dtypes(all_data)

del items
gc.collect()

77

In [29]:
lag_variable = ['target']
lags = [1 ,2 ,3 ,4, 5, 12]
for lag in lags:
    sales_new_df = all_data[['date_block_num','shop_id','item_id']+lag_variable].copy()
    sales_new_df.date_block_num+=lag
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variable]
    all_data = pd.merge(all_data, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')
    del sales_new_df
    gc.collect()

for feat in all_data.columns:
    if 'target' in feat:
        all_data[feat]=all_data[feat].fillna(0)

In [30]:

all_data['avg_sales_per_shop_id'] = 0.0
all_data['avg_sales_per_item_id'] = 0.0
all_data['avg_sales_per_item_cat_id'] = 0.0

avg_shop_sales_per_month = all_data.groupby(['shop_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_sales_per_month = all_data.groupby(['item_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_cat_sales_per_month = all_data.groupby(['item_category_id','date_block_num'],as_index=False).agg({'target':'mean'})

#lets calculate  6 lags
lag_variable = ['item_id','shop_id','item_category_id']
lags = [1 ,2 ,3 ,4, 5, 12]
curr = 0
for lag in lags:
    diff = lag - curr
    curr = lag
    avg_shop_sales_per_month.date_block_num+=diff
    avg_item_cat_sales_per_month.date_block_num+=diff
    avg_item_sales_per_month.date_block_num+=diff
    
    avg_shop_sales_per_month.columns = ['shop_id','date_block_num']+ [lag_variable[1]+'_lag_'+str(lag)]
    avg_item_sales_per_month.columns = ['item_id','date_block_num']+ [lag_variable[0]+'_lag_'+str(lag)]
    avg_item_cat_sales_per_month.columns = ['item_category_id','date_block_num']+ [lag_variable[2]+'_lag_'+str(lag)]
    
    all_data = pd.merge(all_data, avg_shop_sales_per_month,on=['date_block_num','shop_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_sales_per_month,on=['date_block_num','item_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_cat_sales_per_month,on=['date_block_num','item_category_id'] ,how='left')
    
    gc.collect()
del avg_item_cat_sales_per_month,avg_item_sales_per_month,avg_shop_sales_per_month

In [31]:
all_data.drop(columns=['mean'],inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [33]:
all_data.fillna(0,inplace=True)
gc.collect()

394

In [43]:
X_train.target_lag_1.head()

4836102    0.0
4836103    0.0
4836104    1.0
4836105    1.0
4836106    0.0
Name: target_lag_1, dtype: float32

In [42]:
(X_train.target_lag_1 + 1).head()

4836102    1.0
4836103    1.0
4836104    2.0
4836105    2.0
4836106    1.0
Name: target_lag_1, dtype: float32

In [34]:
X_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)].drop(['target','ID'], axis = 1)
y_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)]['target']
X_val =  all_data[all_data['date_block_num']==33].drop(['target','ID'], axis = 1)
y_val = all_data[all_data['date_block_num'] == 33]['target']
X_test = all_data[(all_data['date_block_num']==34)].drop(['target'],axis=1)
del all_data

In [35]:
X_test.sort_values('ID',inplace=True)

In [36]:
gc.collect()

7

In [37]:
X_train.to_pickle(PATH + 'X_train.pkl')
y_train.to_pickle(PATH + 'y_train.pkl')
X_val.to_pickle(PATH + 'X_val.pkl')
y_val.to_pickle(PATH + 'y_val.pkl')
X_test.to_pickle(PATH + 'X_test.pkl')

In [39]:
X_train.dtypes

date_block_num                 int32
item_id                        int32
month                          int32
shop_id                        int32
year                           int32
item_category_id               int32
target_lag_1                 float32
target_lag_2                 float32
target_lag_3                 float32
target_lag_4                 float32
target_lag_5                 float32
target_lag_12                float32
avg_sales_per_shop_id        float64
avg_sales_per_item_id        float64
avg_sales_per_item_cat_id    float64
shop_id_lag_1                float32
item_id_lag_1                float32
item_category_id_lag_1       float32
shop_id_lag_2                float32
item_id_lag_2                float32
item_category_id_lag_2       float32
shop_id_lag_3                float32
item_id_lag_3                float32
item_category_id_lag_3       float32
shop_id_lag_4                float32
item_id_lag_4                float32
item_category_id_lag_4       float32
s

In [38]:
X_train.head()

,date_block_num,item_id,month,shop_id,year,item_category_id,target_lag_1,target_lag_2,target_lag_3,target_lag_4,...,item_category_id_lag_3,shop_id_lag_4,item_id_lag_4,item_category_id_lag_4,shop_id_lag_5,item_id_lag_5,item_category_id_lag_5,shop_id_lag_12,item_id_lag_12,item_category_id_lag_12
4836102,13,30,1,2,1,40,0.0,0.0,0.0,0.0,...,0.244913,0.089066,0.521739,0.236217,0.107034,0.288889,0.227597,0.059868,12.456522,0.266213
4836103,13,31,1,2,1,37,0.0,0.0,0.0,0.0,...,0.183129,0.089066,0.543478,0.170779,0.107034,0.422222,0.198978,0.059868,11.195652,0.196096
4836104,13,32,1,2,1,40,1.0,0.0,0.0,0.0,...,0.244913,0.089066,1.260870,0.236217,0.107034,1.311111,0.227597,0.059868,4.478261,0.266213
4836105,13,33,1,2,1,37,1.0,1.0,2.0,0.0,...,0.183129,0.089066,0.717391,0.170779,0.107034,0.955556,0.198978,0.059868,0.847826,0.196096
4836106,13,34,1,2,1,40,0.0,0.0,0.0,0.0,...,0.244913,0.089066,0.043478,0.236217,0.107034,0.200000,0.227597,0.059868,0.239130,0.266213


## Exploratory Analysis and Feature Engineering

In [ ]:
!ls {PATH}

In [ ]:
sample_subm = pd.read_csv(f'{PATH}sample_submission.csv')
display(sample_subm.tail())

In [ ]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
train = pd.read_csv(f'{PATH}sales_train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [ ]:
test.head()

In [ ]:
# we'll be dropping this shortly, but it will be readded later 
temp_item_price = train['item_price']

In [ ]:
#Make Monthly
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train = train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
#Add Mean Feature
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
#Last Month (Oct 2015)
shop_item_prev_month = train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()
#Add Previous Month Feature
train = pd.merge(train, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
train = pd.merge(train, items, how='left', on='item_id')
#Item Category features
train = pd.merge(train, item_categories, how='left', on='item_category_id')
#Shops features
train = pd.merge(train, shops, how='left', on='shop_id')
train.head()

In [ ]:
test = test.drop('ID', axis=1)

In [ ]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
#Add Mean Feature
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Add Previous Month Feature
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test = pd.merge(test, items, how='left', on='item_id')
#Item Category features
test = pd.merge(test, item_categories, how='left', on='item_category_id')
#Shops features
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

In [ ]:
train['item_price'] = temp_item_price

In [ ]:
price = train.reset_index().set_index(
    ['item_id', 'shop_id', 'date_block_num']).sort_index()[['item_price', 'item_cnt_month']]
price.head()

In [ ]:
# Helpers
def closest_date_block(current_day, item_id, shop_id):
    """Find the block_date which is closest to the current_day, given item_id and shop_id. Returns index integer"""
    if (item_id, shop_id) in price.index:
        search_lst = np.array(price.loc[(item_id, shop_id)].index)        
        return search_lst[np.abs(current_day - search_lst).argmin()]
    return -1
                
def closest_price(current_day, item_id, shop_id):
    closest_date = closest_date_block(current_day, item_id, shop_id)
    if closest_date != -1:
        return price.loc[( item_id, shop_id, closest_date )]['item_price']
    return np.nan

counter = 0
from IPython.display import clear_output
def closest_price_lambda(x):
    global counter
    if counter % 5000 == 0:
        # hack to display progress, obviously this isn't a
        # robust way to do things :)
        clear_output()
        print("{}/{}".format(counter, len(test)))
    counter += 1
    result = closest_price(34, x.item_id, x.shop_id)
    if counter == len(test):
        clear_output()
        print("DONE")
    return result

In [ ]:
closest_prices = test.apply(closest_price_lambda, axis=1)

In [ ]:
test['item_price'] = closest_prices

# Applying Categorical Variables

In [ ]:
cat_vars = [
    'date_block_num',
    'item_category_id',
    'item_category_name',
    'item_id',
    'item_name',
    'month',              
    'shop_id',
    'shop_name',
    'year',               
]
contin_vars = [
    'item_cnt_month_mean',
    'item_cnt_prev_month',
    'item_cnt_month',
    'item_price',
]
dep = 'item_cnt_month'

In [ ]:
date_block_num_cat = pd.api.types.CategoricalDtype(categories=[i for i in range(35)], ordered=True)
item_category_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_id']), ordered=True)
item_category_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_name']), ordered=True)
item_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_id']), ordered=True)
item_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_name']), ordered=True)
month_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['month']), ordered=True)
shop_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_id']), ordered=True)
shop_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_name']), ordered=True)
year_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['year']), ordered=True)

In [ ]:
train['date_block_num'] = train['date_block_num'].astype(date_block_num_cat)
train['item_id'] = train['item_id'].astype(item_id_cat)
train['item_category_id'] = train['item_category_id'].astype(item_category_id_cat)
train['item_category_name'] = train['item_category_name'].astype(item_category_name_cat)
train['item_name'] = train['item_name'].astype(item_name_cat)
train['shop_id'] = train['shop_id'].astype(shop_id_cat)
train['shop_name'] = train['shop_name'].astype(shop_name_cat)
train['month'] = train['month'].astype(month_cat)
train['year'] = train['year'].astype(year_cat)

test['date_block_num'] = test['date_block_num'].astype(date_block_num_cat)
test['item_id'] = test['item_id'].astype(item_id_cat)
test['item_category_id'] = test['item_category_id'].astype(item_category_id_cat)
test['item_category_name'] = test['item_category_name'].astype(item_category_name_cat)
test['item_name'] = test['item_name'].astype(item_name_cat)
test['shop_id'] = test['shop_id'].astype(shop_id_cat)
test['shop_name'] = test['shop_name'].astype(shop_name_cat)
test['month'] = test['month'].astype(month_cat)
test['year'] = test['year'].astype(year_cat)

for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')


In [ ]:
assert not train.isnull().any().any()

In [ ]:
assert not test.drop('item_price', axis=1).isnull().any().any() # item price can be nan by design

In [ ]:
train.to_pickle(PATH + 'train.pkl')
test.to_pickle(PATH + 'test.pkl')

In [ ]:
!ls {PATH}

# Training

## _Deep Learning Approach_

In [ ]:
train = pd.read_pickle(PATH + 'train.pkl')
test = pd.read_pickle(PATH + 'test.pkl')

In [ ]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [ ]:
val = train[train['date_block_num']==33]

In [ ]:
train = train[train['date_block_num']<33]

In [ ]:
df, y, nas, mapper = proc_df(train, dep, do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    val,
    dep,
    do_scale=True,
    mapper=mapper,
)
test_df, _, nas, mapper = proc_df(
    test,
    dep,
    do_scale=True,
    mapper=mapper,
)

In [ ]:
test_df = test_df.drop('item_price_na', axis=1) # ignore these

In [ ]:
y_train = np.log1p(y.clip(0.,40.))
y_val = np.log1p(val_y.clip(0.,40.))

In [ ]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df,
    is_reg=True,  # is regression
    is_multi=False, 
)

In [ ]:
cat_sz = [(c, len(train[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.05,
    out_sz=1,
    szs=[1000, 500],
    drops=[.001, .01],
    use_bn=True
)

In [ ]:
m.lr_find(1e-7)

In [ ]:
m.sched.plot(1)

In [ ]:
lr = 1e-5
m.fit(lrs=lr, n_cycle=3, metrics=[rmse], cycle_len=1, cycle_mult=2)

In [ ]:
m.fit(1e-5, n_cycle=2, metrics=[rmse])

## Submit It!

In [ ]:
result = m.predict(is_test=True)
predictions = np.expm1(result).clip(0, 20)
display(predictions)

In [ ]:
sample_subm.head()

In [ ]:
final_dataframe = pd.read_csv(f'{PATH}test.csv')

In [ ]:
final_dataframe['item_cnt_month'] = predictions
final_dataframe = final_dataframe[['ID', 'item_cnt_month']]

In [ ]:
final_dataframe.head()

In [ ]:
len(final_dataframe)

In [ ]:
len(np.unique(final_dataframe['ID']))

In [ ]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')